# Statistics

In [ ]:
import numpy as np

candidates = np.genfromtxt('.candidates_V2.csv', delimiter=',', dtype=None, skip_header=1)



In [ ]:
print("The total number of scans is " + str(len(candidates)) + ".")
np.unique(candidates, False, False, True, )

The total number of scans is 754975.


# Loading the scan data

In [12]:

import itk

scan_path = "folder"
mhd_data = itk.imread(scan_path)
# convert to numpy array
scan = np.array(itk.GetArrayFromImage(mhd_data), dtype=np.float32)
scan.clip(-1000, 1000, scan)  # clip to reasonable HU values

origin_xyz = mhd_data.GetOrigin()
voxel_size_xyz = mhd_data.GetSpacing()
direction_matrix = np.array(mhd_data.GetDirection()).reshape(3, 3)

voxel_coord = 0 # TODO

patient_coord = origin_xyz + voxel_size_xyz * (direction_matrix @ voxel_coord)


RuntimeError: Could not create IO object for reading file folder
The file doesn't exist. 
Filename = folder